In [1]:
import sys
print(sys.executable)

import cv2
import numpy as np
import os
import glob
import shutil

/home/nathan/miniconda3/envs/milk/bin/python


In [32]:
def get_whitespace(src, threshold=190):
    if len(src.shape) == 3:
        src = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)
    white = src > 215
    dims = src.shape[0] * src.shape[1]
    print(src.shape, white.shape, np.sum(white) / float(dims))
    return white

def maskout_whitespace(white_img, target, values=[0,0,0]):
    if white_img.shape[0] != target.shape[0]:
        print('Shape mismatch; resizing: {} --> {}'.format(white_img.shape, target.shape))
        white_img = cv2.resize((white_img).astype(np.uint8), dsize=(target.shape[1], target.shape[0]), 
                               interpolation=cv2.INTER_NEAREST)
        white_img = white_img.astype(np.bool)
        
    target = np.split(target, 3, axis=-1)
    
    for k in range(3):
        target[k][white_img] = 0
        
    target = np.concatenate(target, axis=-1)
    return target

In [26]:
# white_src = '/home/nathan/Dropbox/_papers/gleason_grade/colorized/validataion_wsi_colored/*colored.jpg'
white_src = '../densenet_small/ext_75pct/inference/*rgb.jpg'
white_src_imgs = sorted(glob.glob(white_src))

white_basenames = [os.path.basename(x).split('_')[0] for x in white_src_imgs]
# white_imgs = [get_whitespace(cv2.imread(x)) for x in white_src_imgs]
white_imgs = {}

for white_img, white_basename in zip(white_src_imgs, white_basenames):
    img = cv2.imread(white_img)
    white_imgs[white_basename] = get_whitespace(img)

(1640, 2988) (1640, 2988) 0.18235462173898848
(2090, 2988) (2090, 2988) 0.2056361010229114
(2074, 3735) (2074, 3735) 0.32867800356036814
(2287, 3610) (2287, 3610) 0.1721931863465305
(2782, 3112) (2782, 3112) 0.1690842387437419
(2749, 2739) (2749, 2739) 0.2017999575271223
(1204, 2054) (1204, 2054) 0.20992100334166863
(2828, 3859) (2828, 3859) 0.19176062277312023
(2618, 2988) (2618, 2988) 0.13070220786379538
(1139, 2054) (1139, 2054) 0.1444322006440676
(2484, 3610) (2484, 3610) 0.2682833290956861
(2527, 3361) (2527, 3361) 0.3035989651543161
(1340, 2054) (1340, 2054) 0.12114040314493743
(2742, 3610) (2742, 3610) 0.11893465957880997
(1457, 2116) (1457, 2116) 0.23229134366003115
(1041, 2054) (1041, 2054) 0.10360609368379405
(2369, 3112) (2369, 3112) 0.16635749250440296
(1339, 2303) (1339, 2303) 0.11230051266053273
(2667, 3984) (2667, 3984) 0.16460385975849404
(2276, 2988) (2276, 2988) 0.18764042696856553
(2641, 3984) (2641, 3984) 0.2756312071458876
(2348, 3112) (2348, 3112) 0.14943199484985

In [34]:
maskout_src = '../densenet_small/ext_75pct/inference/*_color.jpg'
# maskout_src = '../tfhub/inference/mobilenet_v2_050_224_75pct/*color.jpg'
maskout_src_imgs = sorted(glob.glob(maskout_src))

maskout_basenames = [os.path.basename(x).split('_')[0] for x in maskout_src_imgs]
for maskout_img, maskout_basename in zip(maskout_src_imgs, maskout_basenames):
    if maskout_basename not in white_basenames:
        print('{} not in references'.format(maskout_basename))
        continue
        
    dst = maskout_img.replace('.jpg', '_masked.jpg')
    target_img = cv2.imread(maskout_img)
    masked_img = maskout_whitespace(white_imgs[maskout_basename], target_img)
    print(maskout_img, target_img.shape, masked_img.shape, dst)
    
    cv2.imwrite(dst, masked_img)

../densenet_small/ext_75pct/inference/1304700_color.jpg (1640, 2988, 3) (1640, 2988, 3) ../densenet_small/ext_75pct/inference/1304700_color_masked.jpg
../densenet_small/ext_75pct/inference/1304701_color.jpg (2090, 2988, 3) (2090, 2988, 3) ../densenet_small/ext_75pct/inference/1304701_color_masked.jpg
../densenet_small/ext_75pct/inference/1304702_color.jpg (2074, 3735, 3) (2074, 3735, 3) ../densenet_small/ext_75pct/inference/1304702_color_masked.jpg
../densenet_small/ext_75pct/inference/1304703_color.jpg (2287, 3610, 3) (2287, 3610, 3) ../densenet_small/ext_75pct/inference/1304703_color_masked.jpg
../densenet_small/ext_75pct/inference/1304704_color.jpg (2782, 3112, 3) (2782, 3112, 3) ../densenet_small/ext_75pct/inference/1304704_color_masked.jpg
../densenet_small/ext_75pct/inference/1304705_color.jpg (2749, 2739, 3) (2749, 2739, 3) ../densenet_small/ext_75pct/inference/1304705_color_masked.jpg
../densenet_small/ext_75pct/inference/1304706_color.jpg (1204, 2054, 3) (1204, 2054, 3) ../den

../densenet_small/ext_75pct/inference/1305482_color.jpg (1418, 2054, 3) (1418, 2054, 3) ../densenet_small/ext_75pct/inference/1305482_color_masked.jpg
../densenet_small/ext_75pct/inference/1305483_color.jpg (1457, 2054, 3) (1457, 2054, 3) ../densenet_small/ext_75pct/inference/1305483_color_masked.jpg
../densenet_small/ext_75pct/inference/1305484_color.jpg (2652, 3237, 3) (2652, 3237, 3) ../densenet_small/ext_75pct/inference/1305484_color_masked.jpg
../densenet_small/ext_75pct/inference/1305485_color.jpg (1893, 3735, 3) (1893, 3735, 3) ../densenet_small/ext_75pct/inference/1305485_color_masked.jpg
../densenet_small/ext_75pct/inference/1305487_color.jpg (2409, 3237, 3) (2409, 3237, 3) ../densenet_small/ext_75pct/inference/1305487_color_masked.jpg
../densenet_small/ext_75pct/inference/1305488_color.jpg (2617, 3735, 3) (2617, 3735, 3) ../densenet_small/ext_75pct/inference/1305488_color_masked.jpg
../densenet_small/ext_75pct/inference/1305489_color.jpg (2261, 2988, 3) (2261, 2988, 3) ../den